In [0]:
pip install nltk

Python interpreter will be restarted.
Requirement already satisfied: nltk in /databricks/python3/lib/python3.8/site-packages (3.7)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (from nltk) (4.64.0)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from nltk) (8.1.3)
Requirement already satisfied: regex>=2021.8.3 in /databricks/python3/lib/python3.8/site-packages (from nltk) (2022.4.24)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (1.0.1)
Python interpreter will be restarted.

In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pandas as pd
import pickle

ICD9CODES = pickle.load(open("/dbfs/FileStore/data_2/ICD9CODES.p", 'rb'))
ICD9CODES_TOP10 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CODES_TOP10.p", 'rb'))
ICD9CODES_TOP50 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CODES_TOP50.p", 'rb'))
ICD9CAT_TOP10 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CAT_TOP10.p", 'rb'))
ICD9CAT_TOP50 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CAT_TOP50.p", 'rb'))

In [0]:
from pyspark.ml.feature import StopWordsRemover
STOPWORDS_v0 = StopWordsRemover.loadDefaultStopWords("english") + ICD9CODES
STOPWORDS_v0 = [str(i) for i in STOPWORDS_v0]

In [0]:
from pyspark.ml.feature import HashingTF, IDF, RegexTokenizer, StopWordsRemover

def create_TFIDF_v0(trainData, applyData, inputCol="text", outputCol="features", minDocFreq=3, numFeatures=20):    
    tokenizer = RegexTokenizer(pattern="[.:\s]+", inputCol=inputCol, outputCol="z_words")
    wordsData1 = tokenizer.transform(trainData)
    wordsData2 = tokenizer.transform(applyData)
    
    remover = StopWordsRemover(inputCol="z_words", outputCol="z_filtered", stopWords=STOPWORDS_v0)
    wordsDataFiltered1 = remover.transform(wordsData1)
    wordsDataFiltered2 = remover.transform(wordsData2)
    
    hashingTF = HashingTF(inputCol="z_filtered", outputCol="z_rawFeatures", numFeatures=numFeatures)
    featurizedData1 = hashingTF.transform(wordsDataFiltered1)
    featurizedData2 = hashingTF.transform(wordsDataFiltered2)
    # alternatively, CountVectorizer can also be used to get term frequency vectors

    idf = IDF(inputCol="z_rawFeatures", outputCol=outputCol, minDocFreq=minDocFreq)
    idfModel = idf.fit(featurizedData1)
    
    rescaledData = idfModel.transform(featurizedData2)
    return rescaledData.drop("z_words", "z_filtered", "z_rawFeatures", inputCol)

In [0]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

STOPWORDS_v1 = list(ENGLISH_STOP_WORDS) + ICD9CODES

# print "TFIDF v1 stop words"
# print STOPWORDS_v1

In [0]:
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from pyspark.mllib.util import Vectors
from pyspark.mllib.linalg import VectorUDT
from pyspark.sql.functions import UserDefinedFunction

def preprocessor_v1(text):
    text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    return text

def create_TFIDF_v1(df_train, df_apply, inputCol="text", outputCol="features",
                    minDocFreq=3, maxDocFreq=1.0, numFeatures=20):
    df_train['z_cleaned'] = df_train[inputCol].apply(preprocessor_v1)
    df_apply['z_cleaned'] = df_apply[inputCol].apply(preprocessor_v1)

    # Now we create the sparse matrix of tfidf values
    tfidf = TfidfVectorizer(input='content',ngram_range=(1, 1),
                            stop_words=STOPWORDS_v1, 
                            min_df=minDocFreq,
                            max_df=maxDocFreq,
                            max_features=numFeatures)
    # I select to remove stopwords and minimun doc frequency =10 to delete very unusual words
    # that only show up in less than 10 notes (out of 59k notes available) 

    tfidf.fit([c for c in df_train['z_cleaned']])
    dtm = tfidf.transform([c for c in df_apply['z_cleaned']]).tocsr()
    dtm.sort_indices()
    df_apply[outputCol] = list(dtm)
   
    del df_train['z_cleaned']
    del df_apply['z_cleaned']
    del df_apply[inputCol]
    
    return df_apply

<hd5> WORD2VEC

In [0]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')



STOPWORDS_WORD2VEC = stopwords + ICD9CODES

In [0]:
import numpy as np
import re

# Run this cell if you are using Glove type format
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

def preprocessor_word2vec(text):
    text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    #text = gensim.parsing.preprocessing.remove_stopwords(text)
    return text

def makeFeatureVec(words, model, num_features, index2word_set):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    #index2word_set = set(model.wv.index2word) #activate if using gensim

    # activate if uploaded text version
    #index2word_set=set(keys_updated)
    
    
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, index2word_set, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 10000th review
       if counter%10000 == 0:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features,index2word_set)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

def create_WORD2VEC(df, inputCol="text", outputCol="features",
                    word2vecmodel="./data/model_word2vec.txt"):
    df['z_cleaned'] = df[inputCol].apply(preprocessor_word2vec)
    
    # Create tokens
    token_review=[]
    for i in range(df['z_cleaned'].shape[0]):
        review = df['z_cleaned'][i]
        token_review.append([i for i in review.split()])
    model_w2v = word2vecmodel
#     model_w2v = loadGloveModel(word2vecmodel)
    numFeatures = len(model_w2v.values()[0])
    print("numFeatures: ", numFeatures)
    
    keys_updated = [word for word in model_w2v.keys() if word not in STOPWORDS_WORD2VEC]
    index2word_set=set(keys_updated)

    final_w2v = getAvgFeatureVecs(token_review, model_w2v, index2word_set, num_features=numFeatures)
    df[outputCol] = list(final_w2v)
    
    del df['z_cleaned']
    del df[inputCol]
    
    return df

DOC2VEC
Note: This code only converts the data using a pre-trained doc2vec model

In [0]:
def create_DOC2VEC(df,doc2vecmodel):
    import pandas as pd
    import numpy as np
    df1=pd.read_csv(doc2vecmodel, index_col='id') 
    df1['features']=list(df1.values)
    df1=df1['features'].apply(np.asarray)
        
    result = pd.merge(pd.DataFrame({'id':df1.index, 'features':df1.values}), df, on='id')
    del result['text']
    del df1
    
    return result

In [0]:
# df = pd.DataFrame()

HELPER FUNCTION

In [0]:
import random, pickle
import pandas as pd
from pyspark.mllib.util import Vectors
from pyspark.mllib.linalg import VectorUDT
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
import numpy as np

def separate(seed, N):    
    idx=list(range(N))
    random.seed(seed)
    random.shuffle(idx)
    idx_train= idx[0:int(N*0.50)]
    idx_val= idx[int(N*0.50):int(N*0.75)]
    idx_test= idx[int(N*0.75):N]

    return idx_train, idx_val, idx_test

def separate_2(df, hadmid_pickle):
    f = open(hadmid_pickle, 'rb')
    hadmid_train = pickle.load(f)
    hadmid_val = pickle.load(f)
    hadmid_test = pickle.load(f)
    f.close()
    
    df2 = df.copy()
    df2['_idx'] = df2.index
    df2.set_index('id', inplace=True)
    
    idx_train = list(df2.loc[hadmid_train]['_idx'])
    idx_val = list(df2.loc[hadmid_val]['_idx'])
    idx_test = list(df2.loc[hadmid_test]['_idx'])
    
    return idx_train, idx_val, idx_test

def output_to_spark(df, path, col='features', dense=False):
    if type(df) != pd.DataFrame:    
        print('hello')
        udf = UserDefinedFunction(lambda x: Vectors.stringify(x), StringType())
        s_df2 = df.withColumn(col, udf(df[col]))
        # df2.write.csv(path, header=True)
        s_df2.registerTempTable(path)
#         df3 = df2.toPandas()
#         df3.to_csv(path, index=False)
    else:
        N = df[col].iloc[0].shape[-1]
        if dense:
            def to_string(x):
                return "({0},[{1}],[{2}])".format(N, 
                                                  ",".join([str(i) for i in range(N)]),
                                                  ",".join([str(i) for i in list(x)]))
        else:            
            def to_string(x):
                return "({0},[{1}],[{2}])".format(N, 
                                      ",".join([str(i) for i in list(x.indices)]),
                                      ",".join([str(i) for i in list(x.data)]))
        try :
          df2 = df.copy()
          df2[col] = df[col].apply(to_string)
          s_df2 = spark.createDataFrame(df2)
          s_df2.registerTempTable(path)
        except :
          d = 'hi'
        
        
    return 'hi'
def batch_output_spark(df, otype, fname, labels, outputCol='features', 
                     hadmid_pickle='/dbfs/FileStore/data/TRAIN-VAL-TEST-HADMID.p'):
    labels2 = ['id'] + labels + [outputCol]
        
    if otype.lower() == "tfidfv0":
        f = open(hadmid_pickle, 'rb')
        train_id_df = spark.createDataFrame(zip(pickle.load(f)), ['id2'])
        val_id_df = spark.createDataFrame(zip(pickle.load(f)), ['id2'])
        test_id_df = spark.createDataFrame(zip(pickle.load(f)), ['id2'])
        f.close()
        
        df.cache()

        df1 = df.join(train_id_df, train_id_df.id2 == df.id, 'inner').select(labels2)
        output_to_spark(df1, "{0}_train.csv".format(fname))
        df1 = df.join(val_id_df, val_id_df.id2 == df.id, 'inner').select(labels2)
        output_to_spark(df1, "{0}_val.csv".format(fname))
        df1 = df.join(test_id_df, test_id_df.id2 == df.id, 'inner').select(labels2)
        output_to_spark(df1, "{0}_test.csv".format(fname))
        
    elif otype.lower() == "tfidfv1":
        idx_train, idx_val, idx_test = separate_2(df, hadmid_pickle)
        
        output_to_spark(df.loc[idx_train][labels2], "{0}_train".format(fname), dense=False)
        output_to_spark(df.loc[idx_val][labels2], "{0}_val".format(fname), dense=False)
        output_to_spark(df.loc[idx_test][labels2], "{0}_test".format(fname), dense=False)
    elif otype.lower() == "word2vecv0":
        idx_train, idx_val, idx_test = separate_2(df, hadmid_pickle)
        output_to_spark(df.loc[idx_train][labels2], "{0}_train".format(fname), dense=True)
        output_to_spark(df.loc[idx_val][labels2], "{0}_val".format(fname), dense=True)
        output_to_spark(df.loc[idx_test][labels2], "{0}_test".format(fname), dense=True)
    elif otype.lower() == "doc2vecv0":
        # doc2vec has the same format as word2vec
        idx_train, idx_val, idx_test = separate_2(df, hadmid_pickle)
        output_to_spark(df.loc[idx_train][labels2], "{0}_train".format(fname), dense=True)
        output_to_spark(df.loc[idx_val][labels2], "{0}_val".format(fname), dense=True)
        output_to_spark(df.loc[idx_test][labels2], "{0}_test".format(fname), dense=True)
        
def read_csv(path):
    df = spark.read.csv(path, header=True, inferSchema=True)
    
    udf = UserDefinedFunction(lambda x: Vectors.parse(x), VectorUDT())
    new_df = df.withColumn('features', udf(df.features))
    
    return new_df

Actual Data Preprocessing
prepare separation indices (train, validation, test)

In [0]:

df1_sp = spark.sql('''
select distinct 
`row_id` as id,
`0389`,
`2449`,
`25000`,
`2720`,
`2724`,
`2760`,
`2761`,
`2762`,
`2767`,
`2851`,
`2859`,
`2875`,
`3051`,
`311`,
`32723`,
`4019`,
`40390`,
`40391`,
`41071`,
`412`,
`41401`,
`4168`,
`4240`,
`4241`,
`42731`,
`42789`,
`4280`,
`486`,
`49390`,
`496`,
`5070`,
`5119`,
`5180`,
`51881`,
`53081`,
`5845`,
`5849`,
`5859`,
`5990`,
`7742`,
`78552`,
`99592`,
`9971`,
`V053`,
`V1582`,
`V290`,
`V4581`,
`V4582`,
`V5861`,
`V5867`,
`text`
--`clean_text`
from tempdb.hadm_top_50_icd_step_5
''')

df1_pd = df1_sp.toPandas()


In [0]:
# df1_sp_train.head()

In [0]:
# idx_train, idx_val, idx_test = separate_2(df1_pd, '/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p')
# df1_pd_train = df1_pd.loc[idx_train]

# f = open('/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p', 'rb')
# hadmid_train = pickle.load(f)
# hadmid_val = pickle.load(f)
# hadmid_test = pickle.load(f)
# f.close()
    
# hadmid_train_df = spark.createDataFrame(zip(hadmid_train), ['id2'])
# df1_sp_train = df1_sp.join(hadmid_train_df, hadmid_train_df.id2 == df1_sp.id, 'inner')

In [0]:
from time import time
t0 = time()

df2 = create_TFIDF_v1(df1_pd.copy(), df1_pd.copy(), minDocFreq=10, 
                      maxDocFreq=0.8, numFeatures=40000)

In [0]:
# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

In [0]:
df2_new = df2["features"].astype('string')
spf = pd.DataFrame(df2_new).reset_index()
spf.columns = ["id","features"]
s_df2_new = pandas_to_spark(spf)
s_df2_new.registerTempTable("tfidf_output")

/databricks/spark/python/pyspark/sql/pandas/conversion.py:340: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(

In [0]:
%sql
create table tempdb.tfidf_output as (
select * 
from tfidf_output
)


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * 
from tempdb.tfidf_output 
limit 10

id,features
0,"(0, 6326) 0.19197569127830585 (0, 6457) 0.12709624505626282 (0, 6891) 0.1343398770493549 (0, 7214) 0.17485338375166284 (0, 8101) 0.18784796008227986 (0, 8377) 0.14451317971411926 (0, 9269) 0.18513973756449514 (0, 11964) 0.18952783849733856 (0, 13369) 0.425198023609649 (0, 14422) 0.32953273303373176 (0, 14574) 0.14043445709416455 (0, 19734) 0.14030628244306934 (0, 19796) 0.15726548161603893 (0, 23913) 0.16048722624961378 (0, 26751) 0.20617991474114705 (0, 27742) 0.10144601315375221 (0, 28684) 0.2924615481119045 (0, 29172) 0.12376964972817037 (0, 31804) 0.1763568947895499 (0, 33216) 0.1639204751102144 (0, 33510) 0.1663071397755431 (0, 33568) 0.19217516686301844 (0, 34811) 0.20959046214573204 (0, 37257) 0.2092252238208384 (0, 38088) 0.1697131464116581"
1,"(0, 472) 0.03494981129755392 (0, 473) 0.03461530879783161 (0, 476) 0.09827853714763313 (0, 803) 0.0664901586198983 (0, 880) 0.037535011015165765 (0, 881) 0.04956713243635864 (0, 972) 0.06478821463102233 (0, 1075) 0.05245825317996584 (0, 1383) 0.12040765005013673 (0, 1736) 0.04387451320013108 (0, 1737) 0.06723326107008398 (0, 2222) 0.08135655430424316 (0, 2585) 0.04061937149750106 (0, 2683) 0.04549996522556103 (0, 3029) 0.0354799461429996 (0, 3084) 0.07047782446096308 (0, 3316) 0.1103787479722794 (0, 3789) 0.07262523897878886 (0, 4062) 0.07379378196008364 (0, 4157) 0.05644091634813815 (0, 4526) 0.07794759778393513 (0, 5247) 0.04184694349337137 (0, 5387) 0.05806931926989073 (0, 5659) 0.047991758941657095 (0, 5885) 0.09440377053761605 : : (0, 33985) 0.038309253280478364 (0, 34084) 0.2445476435838154 (0, 34159) 0.0954408698669372 (0, 34173) 0.0444224404804769 (0, 34267) 0.061915619071196394 (0, 34770) 0.05209262716294869 (0, 34773) 0.041800619750725874 (0, 34856) 0.16853130061483648 (0, 34989) 0.03585843113489304 (0, 35785) 0.058384368881350686 (0, 36728) 0.05980508046248022 (0, 36880) 0.03607022686608911 (0, 38224) 0.052071718435575544 (0, 38327) 0.03641212922059722 (0, 38391) 0.06870401477617673 (0, 38626) 0.07952319996365825 (0, 38679) 0.0879120490021288 (0, 38786) 0.07178671312830068 (0, 38921) 0.05456231837461163 (0, 39189) 0.06439675377523792 (0, 39210) 0.04110444342513519 (0, 39388) 0.10065612732192088 (0, 39534) 0.05438323076134064 (0, 39627) 0.090586408744646 (0, 39873) 0.045201420954449484"
2,"(0, 5888) 0.07120439311381299 (0, 6262) 0.17680296248242316 (0, 6615) 0.13883521031154508 (0, 6751) 0.09033065828764754 (0, 8058) 0.12970373065673788 (0, 8377) 0.09630785828428405 (0, 8454) 0.18097074698292992 (0, 8814) 0.11425235349286318 (0, 9637) 0.0819478121514124 (0, 9761) 0.08923327523230111 (0, 11393) 0.06402340757353328 (0, 11403) 0.22644240947576236 (0, 11849) 0.11339911415155743 (0, 13312) 0.14341098918247616 (0, 13942) 0.15930055993874473 (0, 14324) 0.282737131675853 (0, 16023) 0.1457064421945257 (0, 16204) 0.10910859348666811 (0, 17058) 0.0750114719352491 (0, 17182) 0.09032515594178649 (0, 17669) 0.113164682212006 (0, 18577) 0.20310909238204403 (0, 18632) 0.16638795325332414 (0, 20984) 0.0940014091329036 (0, 21132) 0.12897629375333539 : : (0, 25135) 0.09686774721837636 (0, 25554) 0.22414019309105174 (0, 25894) 0.15416580160803595 (0, 26128) 0.13903592323587285 (0, 26132) 0.06580744456000476 (0, 28065) 0.12116556843759611 (0, 28551) 0.10782678509613644 (0, 28590) 0.1197931992100308 (0, 28643) 0.05541150851949356 (0, 29803) 0.09811050739024965 (0, 30611) 0.10897279926523737 (0, 31305) 0.14329306458618926 (0, 32392) 0.06153859921228103 (0, 32526) 0.10906682530119317 (0, 32796) 0.16827886983400303 (0, 33606) 0.18162155341648878 (0, 33995) 0.1425776372673248 (0, 34194) 0.06548493891093127 (0, 34309) 0.15240194971192816 (0, 34991) 0.11625815235931988 (0, 36880) 0.07400401848463592 (0, 36924) 0.09479093943113229 (0, 36962) 0.10671703171291232 (0, 38937) 0.2048709670512429 (0, 38990) 0.09466573024136604"
3,"(0, 700) 0.09824024371892649 (0, 3599) 0.05299463721961074 (0, 4560) 0.10655654539030039 (0, 5393) 0.20057949061779506 (0, 6253) 0.06357004462915676 (0, 6837) 0

In [0]:
TFIDFV1_train = spark.sql('''select distinct c.id,     a.`4019`,    a.`2724`,    a.`25000`,    a.`4280`,    a.`41401`,    a.`53081`,    a.`51881`,    a.`42731`,    a.`5849`,    a.`5990`,    c.features from tempdb.hadm_top_50_icd_step_5 a inner join tempdb.hadm_dataset_lookup b  on a.row_id = b.row_id  and b.group_type = 'train' inner join tempdb.tfidf_output c  on c.id = b.row_id'''
)
TFIDFV1_test = spark.sql('''select distinct c.id,     a.`4019`,    a.`2724`,    a.`25000`,    a.`4280`,    a.`41401`,    a.`53081`,    a.`51881`,    a.`42731`,    a.`5849`,    a.`5990`,    c.features from tempdb.hadm_top_50_icd_step_5 a inner join tempdb.hadm_dataset_lookup b  on a.row_id = b.row_id  and b.group_type = 'test' inner join tempdb.tfidf_output c  on c.id = b.row_id''')

TFIDFV1_valid = spark.sql('''select distinct c.id,     a.`4019`,    a.`2724`,    a.`25000`,    a.`4280`,    a.`41401`,    a.`53081`,    a.`51881`,    a.`42731`,    a.`5849`,    a.`5990`,    c.features from tempdb.hadm_top_50_icd_step_5 a inner join tempdb.hadm_dataset_lookup b  on a.row_id = b.row_id  and b.group_type = 'valid' inner join tempdb.tfidf_output c  on c.id = b.row_id''')


In [0]:
TFIDFV1_valid.repartition(1).write.format('com.databricks.spark.csv').save("/dbfs/FileStore/output/DATA_TFIDFV1_HADM_TOP10_valid.csv",header = 'true')
TFIDFV1_train.repartition(1).write.format('com.databricks.spark.csv').save("/dbfs/FileStore/output/DATA_TFIDFV1_HADM_TOP10_train.csv",header = 'true')
TFIDFV1_test.repartition(1).write.format('com.databricks.spark.csv').save("/dbfs/FileStore/output/DATA_TFIDFV1_HADM_TOP10_test.csv",header = 'true')

In [0]:
%sql
select distinct c.id,     a.`4019`,    a.`2724`,    a.`25000`,    a.`4280`,    a.`41401`,    a.`53081`,    a.`51881`,    a.`42731`,    a.`5849`,    a.`5990`,    c.features from tempdb.hadm_top_50_icd_step_5 a inner join tempdb.hadm_dataset_lookup b  on a.row_id = b.row_id  and b.group_type = 'train' inner join tempdb.tfidf_output c  on c.id = b.row_id

In [0]:
'''select distinct c.id,     a.`4019`,    a.`2724`,    a.`25000`,    a.`4280`,    a.`41401`,    a.`53081`,    a.`51881`,    a.`42731`,    a.`5849`,    a.`5990`,    c.featuresfrom tempdb.hadm_top_50_icd_step_5 a inner join tempdb.hadm_dataset_lookup b  on a.row_id = b.row_id  and b.group_type = 'train' inner join tempdb.tfidf_output c  on c.id = b.row_id'''


id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990,features
26,0,0,0,0,0,0,0,0,0,0,"(0, 2220) 0.08021759686129024 (0, 5324) 0.2137304892109997 (0, 5832) 0.1253308067402864 (0, 6285) 0.06598440154526375 (0, 6470) 0.06866590363825427 (0, 7578) 0.15583481691907408 (0, 7971) 0.08703528934137462 (0, 8126) 0.16838502868493765 (0, 9175) 0.07870371221774941 (0, 9761) 0.07693835723508903 (0, 10137) 0.20232927552519023 (0, 10156) 0.09949076221465279 (0, 10580) 0.08303983089264531 (0, 10930) 0.06957066777226481 (0, 10933) 0.06505160818377273 (0, 11007) 0.22562690986102502 (0, 11441) 0.06534841006572388 (0, 11733) 0.12781881676138349 (0, 11924) 0.08112795944746642 (0, 12087) 0.05789843250959277 (0, 12585) 0.10155745557926378 (0, 12744) 0.1177344217372055 (0, 13537) 0.21374495156667092 (0, 14065) 0.06165671346681768 (0, 14973) 0.1105926161958799 : : (0, 23992) 0.05318834559804123 (0, 26132) 0.1134804626436751 (0, 26335) 0.06531265951823466 (0, 26614) 0.11258967627108073 (0, 26751) 0.11847226711085582 (0, 27278) 0.1012718946664369 (0, 28761) 0.22864204668247134 (0, 29094) 0.1520736456887156 (0, 29172) 0.07111881397963607 (0, 29619) 0.15085043068397208 (0, 30234) 0.07004368471622162 (0, 30262) 0.13048023845028295 (0, 30835) 0.2366576370976026 (0, 30956) 0.12040151337355914 (0, 31745) 0.05907264397185294 (0, 31806) 0.12226135991342416 (0, 32392) 0.10611912916152973 (0, 33709) 0.10968343036506044 (0, 33831) 0.08390953758196933 (0, 34026) 0.2119209743559759 (0, 34694) 0.05422000845905553 (0, 35858) 0.11346202961828943 (0, 37655) 0.10229340238638236 (0, 37914) 0.14370274167121866 (0, 39857) 0.11689960594134205"
29,0,0,0,0,0,0,0,0,0,0,"(0, 0) 0.023022960325209986 (0, 472) 0.017093370276187594 (0, 881) 0.024242458451350937 (0, 4902) 0.02082943410443407 (0, 5567) 0.02094055732816615 (0, 6394) 0.02464771095129089 (0, 7224) 0.03417552981572028 (0, 7550) 0.08521429040317441 (0, 7551) 0.18200784634041547 (0, 7661) 0.02809008784046255 (0, 7671) 0.1396840943275125 (0, 7971) 0.024063330616432857 (0, 8026) 0.06667031177064223 (0, 8271) 0.08060431815991756 (0, 8315) 0.05787672893357912 (0, 8325) 0.11811163058279188 (0, 9184) 0.22591634658246415 (0, 9649) 0.01619565239086362 (0, 9664) 0.022390601226739113 (0, 10404) 0.021705330052181365 (0, 10869) 0.043870607698867685 (0, 11265) 0.09816530654834861 (0, 11795) 0.02512098560609127 (0, 11876) 0.02220562533220624 (0, 11963) 0.05487409630629977 : : (0, 33263) 0.024677958027223487 (0, 33294) 0.162095240280951 (0, 33420) 0.03349710008409584 (0, 33831) 0.023199129456413156 (0, 33842) 0.04229733368411202 (0, 34584) 0.0447830398104694 (0, 34806) 0.014902382188244499 (0, 35130) 0.037511171544181855 (0, 35406) 0.034696831447957065 (0, 35614) 0.02447471931607582 (0, 36077) 0.07853298759041806 (0, 36211) 0.03234337173904062 (0, 36223) 0.10788198322608833 (0, 36366) 0.08725372465690535 (0, 36543) 0.060844567243647646 (0, 36752) 0.017612468470004517 (0, 37232) 0.05139438319029361 (0, 37336) 0.22464234206590178 (0, 37341) 0.07873408538480475 (0, 37471) 0.13095755384201077 (0, 38490) 0.406082398787214 (0, 38527) 0.04863988333665341 (0, 38635) 0.0859661309874065 (0, 38636) 0.08752314552450043 (0, 38707) 0.14885640958729104"
65,1,0,0,0,0,0,0,0,0,0,"(0, 0) 0.19149232674170846 (0, 50) 0.026597887181597942 (0, 91) 0.024098574028368396 (0, 130) 0.12689393487217282 (0, 163) 0.09978121041302428 (0, 202) 0.025270573041668656 (0, 236) 0.20467773511484025 (0, 276) 0.026281569233221783 (0, 321) 0.02555491557434213 (0, 375) 0.02636283757280437 (0, 472) 0.017771657075201012 (0, 473) 0.03520313127128711 (0, 635) 0.03288803874240687 (0, 669) 0.05008229249645992 (0, 731) 0.04168360968141573 (0, 803) 0.033809633128155235 (0, 836) 0.03561363876975265 (0, 880) 0.07634482926345354 (0, 1075) 0.05334907695731529 (0, 1118) 0.036422896322532046 (0, 1173) 0.034793093966922053 (0, 1179) 0.03448382195412404 (0, 1267) 0.0723599133572228 (0, 1324) 0.03980190809454781 (0, 1357) 0.08488012389000518 : : (0, 37926) 0.03188788233093987 (0, 38153) 0.0351

original code that I used to split the datasets, I decided to do this in the processing script

In [0]:
# idx_list = list(df2['id'])
# random.seed(1234)
# random.shuffle(idx_list)
# idx_train= idx_list[0:int(len(df2)*0.50)]
# idx_val= idx_list[int(len(df2)*0.50):int(len(df2)*0.75)]
# idx_test= idx_list[int(len(df2)*0.75):len(df2)]
# train_set = df2.iloc[idx_train]
# val_set = df2.iloc[idx_val]
# test_set = df2.iloc[idx_test]

In [0]:
# train_set_split_0 = pd.DataFrame(train_set)
# train_set_split_0 = train_set_split_0[["id","features"]]
# train_set_split_0 = train_set_split_0["features"].astype('string')
# spf = pd.DataFrame(train_set_split_0).reset_index()
# spf.columns = ["id","features"]
# s_train_set_split_0 = pandas_to_spark(spf)
# s_train_set_split_0.registerTempTable("s_train_set_split")

In [0]:
# test_set_split_0 = pd.DataFrame(test_set)
# test_set_split_0 = test_set_split_0[["id","features"]]
# test_set_split_0 = test_set_split_0["features"].astype('string')
# spf_test = pd.DataFrame(test_set_split_0).reset_index()
# spf_test.columns = ["id","features"]
# s_test_set_split_0 = pandas_to_spark(spf_test)
# s_test_set_split_0.registerTempTable("s_test_set_split")

In [0]:
# valid_set_split_0 = pd.DataFrame(val_set)
# valid_set_split_0 = valid_set_split_0[["id","features"]]
# valid_set_split_0 = valid_set_split_0["features"].astype('string')
# spf_valid = pd.DataFrame(valid_set_split_0).reset_index()
# spf_valid.columns = ["id","features"]
# s_valid_set_split_0 = pandas_to_spark(spf_valid)
# s_valid_set_split_0.registerTempTable("s_valid_set_split")

In [0]:
# train_set_split_0 = pd.DataFrame(train_set)
# train_set_split_0 = train_set_split_0[["id","features"]]
# train_set_split_0 = train_set_split_0["features"].astype('string')
# spf = pd.DataFrame(train_set_split_0).reset_index()
# spf.columns = ["id","features"]
# s_train_set_split_0 = pandas_to_spark(spf)
# s_train_set_split_0.registerTempTable("s_train_set_split")

In [0]:
# %sql
# create table tempdb.s_train_set_split(
# select * 
# from s_train_set_split)

num_affected_rows,num_inserted_rows


In [0]:
# %sql
# create table tempdb.s_test_set_split(
# select * 
# from s_test_set_split)

num_affected_rows,num_inserted_rows


In [0]:
# %sql
# create table tempdb.s_valid_set_split(
# select * 
# from s_valid_set_split)

num_affected_rows,num_inserted_rows


In [0]:
# %sql
# create table tempdb.dataset_splits_lookup as (

# select * 
# from (
#       select 
#           *,
#           'valid' as type
#       from tempdb.s_valid_set_split

#       union all 

#       select 
#           *,
#           'test' as type
#       from tempdb.s_test_set_split

#       union all 

#       select 
#           *,
#           'train' as type
#       from tempdb.s_train_set_split
#       ) a
# )

num_affected_rows,num_inserted_rows


In [0]:

# from pyspark.sql.functions import col 
# df = s_df2
# path = "DATA_TFIDFV1_HADM_TOP10"
# if type(df) != pd.DataFrame:    
#     udf = udf(lambda x: Vectors.stringify(x), StringType())
#     sn_df2 = df.withColumn(col, udf(df[col]))
#     # df2.write.csv(path, header=True)
#     sn_df2.registerTempTable(path)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3892041885301671> in <module> 
 3 path = "DATA_TFIDFV1_HADM_TOP10" 
 4 if type ( df ) != pd . DataFrame : 
 ----> 5 udf = udf ( lambda x : Vectors . stringify ( x ) , StringType ( ) ) 
 6 sn_df2 = df . withColumn ( col , udf ( df [ col ] ) ) 
 7 # df2.write.csv(path, header=True) 

 /databricks/spark/python/pyspark/sql/udf.py in __call__ (self, *cols) 
 177 judf = self . _judf
 178 sc = SparkContext . _active_spark_context
 --> 179 return Column ( judf . apply ( _to_seq ( sc , cols , _to_java_column ) ) ) 
 180 
 181 # This function is for improving the online help system in the interactive interpreter. 

 /databricks/spark/python/pyspark/sql/column.py in _to_seq (sc, cols, converter) 
 60 """
 61 if converter : 
 ---> 62 cols = [ converter ( c ) for c in cols ] 
 63 return sc . _jvm . PythonUtils . toSeq ( cols ) 
 64 

 /databricks/spark/python/pyspark/sql/column.py in <listcomp> (.0) 
 60 """
 61 if converter : 
 ---> 62 cols = [ converter ( c ) for c in cols ] 
 63 return sc . _jvm . PythonUtils . toSeq ( cols ) 
 64 

 /databricks/spark/python/pyspark/sql/column.py in _to_java_column (col) 
 44 jcol = _create_column_from_name ( col ) 
 45 else : 
 ---> 46 raise TypeError(
 47 "Invalid argument, not a string or column: " 
 48 "{0} of type {1}. " 

 TypeError : Invalid argument, not a string or column: <function <lambda> at 0x7f7e8669c700> of type <class 'function'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [0]:
# batch_output_spark(s_df2, "tfidfv1", "DATA_TFIDFV1_HADM_TOP10", ICD9CODES_TOP10, hadmid_pickle='/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p')


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-4264083589124388> in <module> 
 ----> 1 batch_output_spark ( s_df2 , "tfidfv1" , "DATA_TFIDFV1_HADM_TOP10" , ICD9CODES_TOP10 , hadmid_pickle = '/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p' ) 

 <command-3876196618696078> in batch_output_spark (df, otype, fname, labels, outputCol, hadmid_pickle) 
 86 
 87 elif otype . lower ( ) == "tfidfv1" : 
 ---> 88 idx_train , idx_val , idx_test = separate_2 ( df , hadmid_pickle ) 
 89 
 90 output_to_spark ( df . loc [ idx_train ] [ labels2 ] , "{0}_train" . format ( fname ) , dense = False ) 

 <command-3876196618696078> in separate_2 (df, hadmid_pickle) 
 24 f . close ( ) 
 25 
 ---> 26 df2 = df . copy ( ) 
 27 df2 [ '_idx' ] = df2 . index
 28 df2 . set_index ( 'id' , inplace = True ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1798 """
 1799 if name not in self . columns : 
 -> 1800 raise AttributeError(
 1801 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1802 jc = self . _jdf . apply ( name ) 

 AttributeError : 'DataFrame' object has no attribute 'copy'

In [0]:
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
s_df2 = spark.createDataFrame(df2)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/types.py in _infer_type (obj) 
 1040 try : 
 -> 1041 return _infer_schema ( obj ) 
 1042 except TypeError : 

 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1066 else : 
 -> 1067 raise TypeError ( "Can not infer schema for type: %s" % type ( row ) ) 
 1068 

 TypeError : Can not infer schema for type: <class 'numpy.ndarray'>

During handling of the above exception, another exception occurred:

 TypeError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1071 try : 
 -> 1072 fields . append ( StructField ( k , _infer_type ( v ) , True ) ) 
 1073 except TypeError as e : 

 /databricks/spark/python/pyspark/sql/types.py in _infer_type (obj) 
 1042 except TypeError : 
 -> 1043 raise TypeError ( "not supported type: %s" % type ( obj ) ) 
 1044 

 TypeError : not supported type: <class 'numpy.ndarray'>

The above exception was the direct cause of the following exception:

 TypeError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/types.py in _infer_type (obj) 
 1040 try : 
 -> 1041 return _infer_schema ( obj ) 
 1042 except TypeError : 

 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1073 except TypeError as e : 
 -> 1074 raise TypeError ( "Unable to infer the type of the field {}." . format ( k ) ) from e
 1075 return StructType ( fields ) 

 TypeError : Unable to infer the type of the field data.

During handling of the above exception, another exception occurred:

 TypeError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1071 try : 
 -> 1072 fields . append ( StructField ( k , _infer_type ( v ) , True ) ) 
 1073 except TypeError as e : 

 /databricks/spark/python/pyspark/sql/types.py in _infer_type (obj) 
 1042 except TypeError : 
 -> 1043 raise TypeError ( "not supported type: %s" % type ( obj ) ) 
 1044 

 TypeError : not supported type: <class 'scipy.sparse.csr.csr_matrix'>

The above exception was the direct cause of the following exception:

 TypeError Traceback (most recent call last)
 <command-3892041885301654> in <module> 
 ----> 1 s_df2 = spark . createDataFrame ( df2 ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 718 if has_pandas and isinstance ( data , pandas . DataFrame ) : 
 719 # Create a DataFrame from pandas DataFrame. 
 --> 720 return super(SparkSession, self).createDataFrame(
 721 data, schema, samplingRatio, verifySchema)
 722 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 349 raise 
 350 data = self . _convert_from_pandas ( data , schema , timezone ) 
 --> 351 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 352 
 353 def _convert_from_pandas ( self , pdf , schema , timezone ) : 

 /databricks/spark/python/pyspark/sql/session.py in _create_dataframe (self, data, schema, samplingRatio, verifySchema) 
 752 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 753 else : 
 --> 754 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 755 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 756 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 528 write temp files . 
 529 """
 --> 530 data , schema = self . _wrap_data_schema ( data , schema ) 
 531 return self . _sc . parallelize ( data ) , schema
 532 

 /databricks/spark/python/pyspark/sql/session.py 

In [0]:
s_df2 = pandas_to_spark(df2)

In [0]:
# batch_output_spark(s_df2, "word2vecv0", "DATA_WORD2VECV1_HADM_TOP50", ICD9CODES_TOP50,
#                  hadmid_pickle='/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p')
# batch_output_csv(df2, "word2vecv0", "/dbfs/FileStore/data/DATA_WORD2VECV1_HADM_TOP10CAT", ICD9CAT_TOP10,
#                  hadmid_pickle='/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p')
# batch_output_csv(df2, "word2vecv0", "/dbfs/FileStore/data/DATA_WORD2VECV1_HADM_TOP50CAT", ICD9CAT_TOP50,
#                  hadmid_pickle='/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p')

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-4264083589124389> in <module> 
 ----> 1 batch_output_spark(s_df2, "word2vecv0", "DATA_WORD2VECV1_HADM_TOP50", ICD9CODES_TOP50,
 2 hadmid_pickle='/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p')
 3 # batch_output_csv(df2, "word2vecv0", "/dbfs/FileStore/data/DATA_WORD2VECV1_HADM_TOP10CAT", ICD9CAT_TOP10, 
 4 # hadmid_pickle='/dbfs/FileStore/data/TRAIN_VAL_TEST_HADMID.p') 
 5 # batch_output_csv(df2, "word2vecv0", "/dbfs/FileStore/data/DATA_WORD2VECV1_HADM_TOP50CAT", ICD9CAT_TOP50, 

 <command-3876196618696078> in batch_output_spark (df, otype, fname, labels, outputCol, hadmid_pickle) 
 87 output_to_spark ( df . loc [ idx_test ] [ labels2 ] , "{0}_test" . format ( fname ) , dense = False ) 
 88 elif otype . lower ( ) == "word2vecv0" : 
 ---> 89 idx_train , idx_val , idx_test = separate_2 ( df , hadmid_pickle ) 
 90 output_to_spark ( df . loc [ idx_train ] [ labels2 ] , "{0}_train" . format ( fname ) , dense = True ) 
 91 output_to_spark ( df . loc [ idx_val ] [ labels2 ] , "{0}_val" . format ( fname ) , dense = True ) 

 <command-3876196618696078> in separate_2 (df, hadmid_pickle) 
 24 f . close ( ) 
 25 
 ---> 26 df2 = df . copy ( ) 
 27 df2 [ '_idx' ] = df2 . index
 28 df2 . set_index ( 'id' , inplace = True ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1798 """
 1799 if name not in self . columns : 
 -> 1800 raise AttributeError(
 1801 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1802 jc = self . _jdf . apply ( name ) 

 AttributeError : 'DataFrame' object has no attribute 'copy'

end of original processing code

In [0]:
import gensim

In [0]:
model_w2v = gensim.models.Word2Vec.load('/dbfs/FileStore/data/model_word2vec_skipgram_300dim')

In [0]:
df2 = create_WORD2VEC(df1_pd.copy(),word2vecmodel= model_w2v)

Error while obtaining a new communication channel